<a href="https://colab.research.google.com/github/moosakazim12/SQL-Practice/blob/main/SQL_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Question (SELF JOIN)
# How can you output a list of all members who have recommended another member? Ensure that there are no duplicates in the list, and that results are ordered by (surname, firstname).

SELECT DISTINCT
       m.firstname,
       m.surname
FROM cd.members m
INNER JOIN cd.members r
  ON m.memid = r.recommendedby
ORDER BY m.surname, m.firstname;


In [ ]:
# Question
# How can you produce a list of the start times for bookings by members named 'David Farrell'?
# I have taken tables and question from https://pgexercises.com/questions/joins/simplejoin.html

SELECT starttime
FROM cd.members AS m
INNER JOIN cd.bookings AS b
USING(memid)
WHERE m.firstname = 'David' AND m.surname = 'Farrell'